In [1]:
import os
import json
import pandas as pd

In [2]:
train_path = '/home/bruno/git/hmc-torch/results/train'

In [3]:
#hpo_path = r'C:\Users\bruno\git\hmc-torch\results\hpo\local_constraint'

In [4]:
datasets = ["cellcycle_GO", "derisi_GO", "eisen_GO", "expr_GO", "gasch1_GO", "gasch2_GO", "spo_GO", "seq_GO", "seq_FUN", "cellcycle_FUN", "derisi_FUN", "eisen_FUN", "expr_FUN", "gasch1_FUN", "gasch2_FUN", "spo_FUN"]


In [5]:
datasets[1]

'derisi_GO'

In [6]:
len(datasets)

16

In [7]:
def __load_json__(path):
    with open(path, "r") as f:
        tmp = json.loads(f.read())
    return tmp

In [8]:
def get_results_file(dataset_name):
    local_path = os.path.join(train_path, "local", dataset_name)
    global_path = os.path.join(train_path, "global", f"global-{dataset_name}")
    local_train_path = os.path.join(local_path, os.listdir(local_path)[0])
    global_train_path = os.path.join(global_path, os.listdir(global_path)[0])
    job_id = local_train_path.split("_")[-2:]
    job_id = "_".join(job_id)
    local_json_file_path = os.path.join(local_train_path, "test_scores.json")
    global_json_file_path = os.path.join(global_train_path, "test-scores.json")
    return local_json_file_path, global_json_file_path
    

In [9]:
get_results_file("seq_FUN")

('/home/bruno/git/hmc-torch/results/train/local/seq_FUN/job_20260111_114835/test_scores.json',
 '/home/bruno/git/hmc-torch/results/train/global/global-seq_FUN/test_20260111_155605/test-scores.json')

In [32]:
def get_score_values(datasets):
    datasets_params = {}
    all_datasets = []
    all_methods = []
    all_precisions = []
    all_recalls = []
    all_f1s = []
    all_avg_precisions = []
    for dataset_name in datasets:
        local_json_file, global_json_file = get_results_file(dataset_name)
        print(local_json_file)
        with open(local_json_file, 'r') as f:
            local_data = json.load(f)

        local_data_global_score = local_data['global']
        local_global_precision = local_data_global_score['precision']
        local_global_recall = local_data_global_score['recall']
        local_global_f1 = local_data_global_score['f1score']
        local_global_avg_precision = local_data_global_score['avgscore']
        
        with open(global_json_file, 'r') as f:
            global_data = json.load(f)
        # Cria listas para cada hiperparâmetro

        print(global_data)

        local_data_global_score = global_data['global']
        global_global_precision = local_data_global_score['precision']
        global_global_recall = local_data_global_score['recall']
        global_global_f1 = local_data_global_score['f1score']
        global_global_avg_precision = local_data_global_score['avg_precision']
       
        all_methods.append('local')
        all_datasets.append(dataset_name)
        all_precisions.append(local_global_precision)
        all_recalls.append(local_global_recall)
        all_f1s.append(local_global_f1)
        all_avg_precisions.append(local_global_avg_precision)
        
        all_methods.append('global')
        all_datasets.append(dataset_name)
        all_precisions.append(global_global_precision)
        all_recalls.append(global_global_recall)
        all_f1s.append(global_global_f1)
        all_avg_precisions.append(global_global_avg_precision)

    # Formata para salvar compatível com YAML de exemplo
    datasets_params['method'] = all_methods
    datasets_params['dataset'] = all_datasets
    datasets_params['precision'] = all_precisions
    datasets_params['recall'] = all_recalls
    datasets_params['f1'] = all_f1s
    datasets_params['avg_precision'] = all_avg_precisions

    df = pd.DataFrame(datasets_params)
    return datasets_params, df

In [33]:
datasets_params, df = get_score_values(datasets)

/home/bruno/git/hmc-torch/results/train/local/cellcycle_GO/job_20260111_183755/test_scores.json
{'0': {'f1score': 1.0, 'precision': 1.0, 'recall': 1.0}, '1': {'f1score': 0.6730979041056561, 'precision': 0.5591490173630986, 'recall': 0.845377181595269}, '2': {'f1score': 0.553068167499685, 'precision': 0.4892984542211653, 'recall': 0.6359509320969767}, '3': {'f1score': 0.37790729342556784, 'precision': 0.39974648536529156, 'recall': 0.3583307509554798}, '4': {'f1score': 0.23107569721115537, 'precision': 0.35313335313335315, 'recall': 0.17172154823801272}, '5': {'f1score': 0.1198443579766537, 'precision': 0.28785046728971964, 'recall': 0.07567567567567568}, '6': {'f1score': 0.09696969696969697, 'precision': 0.28936170212765955, 'recall': 0.058244111349036405}, '7': {'f1score': 0.10095846645367412, 'precision': 0.23511904761904762, 'recall': 0.06427990235964198}, '8': {'f1score': 0.04355108877721943, 'precision': 0.17105263157894737, 'recall': 0.02495201535508637}, '9': {'f1score': 0.01709

In [36]:
df.to_csv('/home/bruno/git/hmc-torch/results/summary_results.csv', index=False)